# Finding best leakage using Bayesian optimization
First lets get some basic analogy out of the way:

In general we can shape the goal of my thesis using the following variables:

$$ Alice = X \sim \rm I\!D^1 $$

$$ Dataset = Y \sim \rm I\!D^k$$ 
such that
$$X \subset Y $$
$$1 < k \le \infty$$

Where $\rm I\!D$ is any distribution

By any given privacy preserving mechanism:

$$f: \rm I\!R^d -> \rm I\!R^n$$ 

$$Z = f(Y)$$

With all this annotation out of the way we can start to look at the goal of my thesis:

$$\underset{Y}{\operatorname{argmax}} q(X,Z)$$

where $q$ is any metric that measures the leakage in a single scalar value


### For this example:

I will:
- Maximize the leakage using BayesianOptimization `GPyOpt`
    - https://gpyopt.readthedocs.io/en/latest/
- Use Mutual Information as $q$


### Imports


In [34]:
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_regression
import pymc3 as pm
import numpy as np
from GPyOpt.methods import BayesianOptimization
import arviz as az
import theano
import theano.tensor as tt
import pandas as pd
import opendp.smartnoise.core as sn

### Defining f using openDP

In [35]:
@theano.compile.ops.as_op(itypes=[tt.lvector,tt.lvector,tt.lvector,
                                  tt.lvector,tt.dvector,tt.lvector,
                                  tt.lscalar],
                          otypes=[tt.dscalar])
def f(age,sex,educ,race,income,married,N):
    temp_file='temp.csv'    
    var_names = ["age", "sex", "educ", "race", "income", "married"]
    data = {
        "age":     age,
        "sex":     sex,
        "educ":    educ,
        "race":    race,
        "income":  income,
        "married": married
    }
    df = pd.DataFrame(data,columns=var_names)
    df.to_csv(temp_file)
    with sn.Analysis() as analysis:
        # load data
        data = sn.Dataset(path=temp_file,column_names=var_names)

        # get mean of age
        age_mean = sn.dp_mean(data = sn.to_float(data['income']),
                              privacy_usage = {'epsilon': .1},
                              data_lower = 0., # min income
                              data_upper = 200., # max income                   
                              data_rows = N
                             )
    analysis.release()
    return np.float64(age_mean.value) 

### Defining q

In [47]:
def q(X,Z):
    print(X)
    mutual_info = 0
    for x in X:
        mutual_info += mutual_info_regression(x.reshape(-1,1),Z)[0]
    return mutual_info

### What is bayesian optimization doing
It will search through each dimensions of our space of unknowns searching for the values which has the highest probability of being the maximum:

Personally I like the explanation from within here:

https://www.cse.wustl.edu/~garnett/cse515t/spring_2015/files/lecture_notes/12.pdf 


### Since we are optimizing real numbers, we need to have a way of converting real numbers into distributions:


In [137]:
def map_int_to_discrete_dist(name, info, shape):
    ids, mu, std = info
    if ids == 0:
        f_a = lambda b: 2*mu-b
        b = 1/3*(mu+2*np.sqrt(3)*np.sqrt(std))
        a = f_a(b)
        if a > b:
            b,a = a,b
        elif a == b:
            b += 1
        print("uniform", a, b)
        return pm.DiscreteUniform(name, int(a), int(b), shape=shape)
    elif ids == 1:
        mu = abs(mu)
        print("posson", mu)
        return pm.Poisson(name, mu=mu, shape=shape)

In [138]:
def map_int_to_cont_dist(name, info, shape):
    ids, mu, std = info
    if ids == 0:
        ##Normal
        return pm.Normal(name, mu, std, shape=shape)
    elif ids == 1:
        #Uniform
        f_a = lambda b: 2*mu-b
        b = 1/3*(mu+2*np.sqrt(3)*np.sqrt(std))
        a = f_a(b)
        if a > b:
            b,a = a,b
        elif a == b:
            b += 1
        print("uniform", a,b)
        return pm.Uniform(name, a, b, shape=shape)
    elif ids == 2:
        #Half Normal
        print("halft", std)
        return pm.HalfNormal(name, std, shape=shape)
    elif ids == 3:
        #Gamma
        print("gamma", mu, std)
        return pm.Gamma(name, mu=mu, sigma=std, shape=shape)

### Now we can construct the method which we are trying to find the maximum of

It will take a vector as input X

In [139]:
def analyse(X):
    print(X)
    ids_cont = X[:,0]
    ids_disc = X[:,1:6][0]
    
    mus_cont = X[:,6]
    mus_disc = X[:,7:12][0]
    
    std_cont = X[:,12]
    std_disc = X[:,13:18][0]
    print(f"ids: {ids_cont}, ids_disc: {ids_disc}, mus_cont: {mus_cont}, mus_disc: {mus_disc}, std_cont {std_cont}, std_dist {std_disc}")
    age_information = [ids_disc[0], mus_disc[0], std_disc[0]]
    sex_information = [ids_disc[1], mus_disc[1], std_disc[1]]
    educ_information = [ids_disc[2], mus_disc[2], std_disc[2]]
    race_information = [ids_disc[3], mus_disc[3], std_disc[3]]
    income_information = [ids_cont[0], mus_cont[0], std_cont[0]]
    married_information = [ids_disc[4], mus_disc[4], std_disc[4]]
    
    with pm.Model() as model:
        N = 10
        N_rv = pm.Constant("N", N)
        
        age_dist = map_int_to_discrete_dist("age", age_information, N)
        sex_dist = map_int_to_discrete_dist("sex", sex_information, N)
        educ_dist = map_int_to_discrete_dist("educ", educ_information, N)
        race_dist = map_int_to_discrete_dist("race", race_information, N)
        income_dist = map_int_to_cont_dist("income", income_information, N)
        married_dist = map_int_to_discrete_dist("married", married_information, N)
        
        out = pm.Deterministic("out", f(age_dist, sex_dist, educ_dist, race_dist, income_dist, married_dist, N_rv))
        trace = pm.sample(1000, return_inferencedata=False, cores=1)
        
        names = ["age", "sex", "educ", "race", "income", "married"]
        alice_metric = np.asarray([trace[n][:,0] for n in names])
        
        I = q(alice_metric, trace["out"])
        return -I

In [140]:
number_of_cont_rv = 1
number_of_disc_rv = 5

domain = [
    {"name": "ids_continuous", "type": "discrete", "domain": (0,1,2,3), "dimensionality": number_of_cont_rv},
    {"name": "ids_continuous", "type": "discrete", "domain": (0,1), "dimensionality": number_of_disc_rv},
    {"name": "mus_cont", "type": "continuous", "domain": (-1000,1000), "dimensionality": number_of_cont_rv},
    {"name": "mus_disc", "type": "discrete", "domain": tuple(range(-1000,1000)), "dimensionality": number_of_disc_rv},
    {"name": "std_cont", "type": "continuous", "domain": (1e-3, 10_000), "dimensionality": number_of_cont_rv},
    {"name": "std_disc", "type": "continuous", "domain": (1e-3, 10_000), "dimensionality": number_of_disc_rv}
]
Bopt = BayesianOptimization(f=analyse, domain=domain)
Bopt.run_optimization(max_iter = 2,eps=1e-6)
print("="*20)
print("Value of (x,y) that minimises the objective:"+str(Bopt.x_opt))    
print("Minimum value of the objective: "+str(Bopt.fx_opt))     
print("="*20)
Bopt.plot_convergence()

[[ 2.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00 -3.45003734e+02 -4.22000000e+02
   6.74000000e+02 -4.14000000e+02  9.20000000e+02 -9.32000000e+02
   9.41867445e+03  6.79305421e+03  2.95968512e+03  3.76994304e+03
   2.82545217e+03  4.05609934e+03]]
ids: [2.], ids_disc: [1. 0. 0. 1. 0.], mus_cont: [-345.00373399], mus_disc: [-422.  674. -414.  920. -932.], std_cont [9418.67445071], std_dist [6793.05421167 2959.68511657 3769.9430428  2825.45216867 4056.09934074]
posson 422.0
uniform 287.4858263402559 1060.514173659744
uniform -760.8984536060632 -67.10154639393687
posson 920.0
halft 9418.674450707105
uniform -1626.8733391418978 -237.12666085810216


Sequential sampling (2 chains in 1 job)
CompoundStep
>CompoundStep
>>Metropolis: [married]
>>Metropolis: [race]
>>Metropolis: [educ]
>>Metropolis: [sex]
>>Metropolis: [age]
>>Metropolis: [N]
>NUTS: [income]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 27 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.


[[  432.           433.           433.         ...   430.
    430.           431.        ]
 [  403.           403.           459.         ...   694.
    694.           694.        ]
 [ -394.          -460.          -226.         ...  -412.
   -412.          -411.        ]
 [  941.           941.           941.         ...   841.
    842.           841.        ]
 [ 1323.70130016  1769.82571602  3482.93450808 ...  4436.04685833
   2020.94247877  1199.20941529]
 [-1538.         -1538.         -1538.         ...  -860.
   -860.          -860.        ]]
[[ 1.00000000e+00  1.00000000e+00  0.00000000e+00  1.00000000e+00
   1.00000000e+00  0.00000000e+00  8.83650849e+02  4.82000000e+02
  -6.33000000e+02  9.39000000e+02 -8.23000000e+02 -2.70000000e+01
   4.83377216e+03  3.83371087e+03  1.94038892e+03  5.09317078e+03
   4.70272687e+03  7.51862857e+03]]
ids: [1.], ids_disc: [1. 0. 1. 1. 0.], mus_cont: [883.65084947], mus_disc: [ 482. -633.  939. -823.  -27.], std_cont [4833.77215757], std_dist [3

Sequential sampling (2 chains in 1 job)
CompoundStep
>CompoundStep
>>Metropolis: [married]
>>Metropolis: [race]
>>Metropolis: [educ]
>>Metropolis: [sex]
>>Metropolis: [age]
>>Metropolis: [N]
>NUTS: [income]


...

[[ -153.          -189.          -211.         ...  -532.
   -532.          -532.        ]
 [   41.            41.            41.         ...   -35.
    -36.           -36.        ]
 [ -544.          -785.          -443.         ...  -868.
   -868.          -868.        ]
 [  232.           232.           232.         ...   205.
    206.           207.        ]
 [-1060.11583457  -609.66357406 -1536.71959642 ... -1550.10731292
  -1109.07411389  -717.87412157]
 [  490.           490.           500.         ...   506.
    507.           507.        ]]
[[ 0.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00 -3.93396371e+02  7.01000000e+02
   5.81000000e+02  2.19000000e+02 -7.57000000e+02 -4.70000000e+02
   1.83974233e+03  4.96988289e+03  8.61188994e+03  4.06969036e+03
   6.20477142e+03  5.76884387e+03]]
ids: [0.], ids_disc: [1. 1. 1. 0. 0.], mus_cont: [-393.39637062], mus_disc: [ 701.  581.  219. -757. -470.], std_cont [1839.74233493], std_dist [

Sequential sampling (2 chains in 1 job)
CompoundStep
>CompoundStep
>>Metropolis: [married]
>>Metropolis: [race]
>>Metropolis: [educ]
>>Metropolis: [sex]
>>Metropolis: [age]
>>Metropolis: [N]
>NUTS: [income]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 25 seconds.
The estimated number of effective samples is smaller than 200 for some parameters.


[[  671.           665.           665.         ...   712.
    712.           712.        ]
 [  611.           611.           611.         ...   570.
    570.           571.        ]
 [  203.           204.           201.         ...   227.
    227.           228.        ]
 [ -992.          -992.          -992.         ...  -747.
   -747.          -747.        ]
 [ -328.44733747   325.71207114   724.33195767 ... -2930.92149086
   1875.44932883 -2673.30855365]
 [ -723.          -723.          -739.         ...  -433.
   -433.          -431.        ]]
[[ 3.00000000e+00  0.00000000e+00  1.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00 -5.21784849e+02 -8.14000000e+02
   6.18000000e+02 -6.33000000e+02 -8.47000000e+02  7.52000000e+02
   3.68147525e+03  4.60455552e+03  3.87818924e+03  9.54838036e+03
   8.58063996e+03  8.72523168e+02]]
ids: [3.], ids_disc: [0. 1. 1. 0. 0.], mus_cont: [-521.78484864], mus_disc: [-814.  618. -633. -847.  752.], std_cont [3681.47524512], std_dist [

SamplingError: Initial evaluation of model at starting point failed!
Starting values:
{'N': array(10), 'age': array([-814, -814, -814, -814, -814, -814, -814, -814, -814, -814]), 'sex': array([618, 618, 618, 618, 618, 618, 618, 618, 618, 618]), 'educ': array([633, 633, 633, 633, 633, 633, 633, 633, 633, 633]), 'race': array([-847, -847, -847, -847, -847, -847, -847, -847, -847, -847]), 'income_log__': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'married': array([751, 751, 751, 751, 751, 751, 751, 751, 751, 751])}

Initial evaluation results:
N                0.00
age            -71.26
sex            -41.32
educ           -41.44
race           -72.03
income_log__      NaN
married        -68.42
Name: Log-probability of test_point, dtype: float64

In [125]:
1e-3

0.001

In [71]:
pm.Uniform.dist(mu=10,std=100)

TypeError: __init__() got an unexpected keyword argument 'mu'

In [135]:
abs(-1)

1